In [ ]:
#%pip install yfinance numpy statsmodels pandas matplotlib arch pycop

In [2]:
import yfinance as yf
import pandas as pd

# List of ticker symbols for each index
tickers = ["^GDAXI", "^AEX", "^N225", "^GSPC", "^GSPTSE"]  # DAX, AEX, Nikkei 225, S&P 500, TSX

# Initialize a DataFrame to store merged data
merged_data = pd.DataFrame()

# Loop over each ticker symbol to fetch the data
for ticker in tickers:
    # Create ticker object
    index = yf.Ticker(ticker)
    
    # Retrieve the historical data starting from November 20, 1992, rescaled by a factor of 0.01
    index_data = index.history(start="1992-11-20") * 0.01
    
    # Extract only the Date and Close columns, reset the index for Date as a column
    close_data = index_data[['Close']].reset_index()
    
    # Convert the Date column to date only (ignoring the time)
    close_data['Date'] = pd.to_datetime(close_data['Date'].dt.date)
    
    # Rename the columns to avoid conflicts in the merge
    close_data = close_data.rename(columns={'Close': ticker})
    
    # Merge with the previously merged data on the 'Date' column
    if merged_data.empty:
        merged_data = close_data
    else:
        merged_data = pd.merge(merged_data, close_data, on='Date', how='outer')

merged_data_filled = merged_data.ffill()



In [ ]:
from arch import arch_model
import numpy as np
import pandas as pd
from pycop import StudentTCopula





def fit_garch(ts):
    model = arch_model(ts, vol="Garch", p=1, q=1, dist="t")
    fitted = model.fit(disp="off")
    residuals = ts - fitted.conditional_volatility  # Approximation for simplicity
    nu = fitted.params["nu"]
    return residuals, nu


# Step 1: Transform the data to uniform margins using ECDF (Empirical CDF)
uniform_data = pd.DataFrame()


# Use empirical CDF for each column (ticker) in the merged data
for ticker in tickers:
    uniform_data[ticker] = merged_data_filled[ticker].rank() / len(merged_data_filled[ticker])

# Step 2: Fit a copula (using Gaussian copula as an example)
copula = StudentTCopula()

# Step 3: Fit the copula to the uniform-transformed data
copula.fit(uniform_data)

# Step 4: Simulate synthetic data from the fitted copula (optional)
simulated_data = copula.sample(len(merged_data_filled))

# Convert the uniform data back to the original scale (inverse of the transformation)
simulated_data_original_scale = pd.DataFrame()

for idx, ticker in enumerate(tickers):
    # Inverse transform (since we used ECDF, we can approximate the inverse by using the quantiles)
    simulated_data_original_scale[ticker] = np.percentile(merged_data_filled[ticker], simulated_data[:, idx] * 100)

# View the simulated data
print(simulated_data_original_scale.head())




marginal_results = [fit_garch(merged_data_filled[i]) for i in tickers]
residuals_list, nu_list = zip(*marginal_results)




ImportError: cannot import name 'StudentTCopula' from 'pycopula.copula' (c:\Programming\Risk-Management-Project\.venv\Lib\site-packages\pycopula\copula.py)

In [6]:
%pip install pycop

Note: you may need to restart the kernel to use updated packages.
